### Solutions

#### Question 1

Write a custom class that will be used to model a single bank account.

Your class should implement functionality to:
- allow initialization with values for `first_name`, `last_name`, `account_number`, `balance`, `is_overdraft_allowed`
- keep track of a "ledger" that keeps a record all transactions (just use a list to keep track of this)
    - at a minimum it should keep track of the transaction date (the current UTC datetime) and the amount (positive, or negative to indicate deposits/withdrawals) - later you could add tracking the running balance as well.
- provide read-only properties for `first_name`, `last_name`, `account_number` and `balance`
- provide a property to access the ledger in such a way that a user of this class cannot mutate the ledger directly
- provide a read-write property for `is_overdraft_allowed` that indicates whether overdrafts are allowed on the account.
- provide methods to debit (`def withdraw`) and credit (`def deposit`) transactions that:
    - verify withdrawals against available balance and `is_overdraft_allowed` flag
        - if withdrawal is larger than available balance and overdrafts are not allowed, this should raise a custom `OverdraftNotAllowed` exception.
        - if transaction value is not positive, this should raise a `ValueError` exception (we have separate methods for deposits and withdrawals, and we expect the value to be positive in both cases - one will add to the balance, one will subtract from the balance).
    - add an entry to the ledger with a current UTC timestamp (positive or negative to indicate credit/debit)
    - keeps the available balance updated
- implements a good string representation for the instance (maybe something like `first_name last_name (account_number): balance`

Feel free to expand on the minimum definition I have given here and enhance your custom class.

#### Question 2

Expand on your class above to implement equality (`==`) comparisons between instances of your class.

Two accounts should be considered equal if the account numbers are the same.

In [1]:
from datetime import datetime

class OverdraftNotAllowed(Exception):
    """Exception indicating a transaction would have resulted in a forbidden overdraft."""


class BankAccount:
    def __init__(self, first_name, last_name, account_number, balance=0, is_overdraft_allowed=False):
        self._first_name = first_name
        self._last_name = last_name
        self._account_number = account_number
        self._balance = balance
        self.is_overdraft_allowed = is_overdraft_allowed
        self._ledger = []
        self._make_ledger_entry(balance, "account creation")
    
    def __str__(self):
        return f"{self.first_name} {self.last_name} ({self._account_number}): {self.balance}"

    def __repr__(self):
        return f"BankAccount(first_name='{self.first_name}', last_name='{self.last_name}', account_number='{self._account_number}', balance={self.balance}, is_overdraft_allowed={self.is_overdraft_allowed})"
    
    def __eq__(self, other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

    @property
    def first_name(self):
        return self._first_name
    
    @property
    def last_name(self):
        return self._last_name
    
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def ledger(self):
        # return a copy of the ledger so it cannot be modified from outside
        return tuple(self._ledger)  

    @property
    def is_overdraft_allowed(self):
        return self._is_overdraft_allowed
    
    @is_overdraft_allowed.setter
    def is_overdraft_allowed(self, value):
        if not isinstance(value, bool):
            raise TypeError("is_overdraft_allowed must be a boolean")
        self._is_overdraft_allowed = value

    def _make_ledger_entry(self, amount, description):
        self._ledger.append(f"{datetime.now()} {description} {amount}; balance {self.balance}")
        print(self.ledger)

    def withdraw(self, amount):
        if amount < 0:
            raise ValueError("amount must be positive")
        if amount > self.balance and not self.is_overdraft_allowed:
            raise OverdraftNotAllowed("amount must be less than or equal to balance")
        self._balance -= amount
        self._make_ledger_entry(-amount, "withdraw")

    def deposit(self, amount):
        if amount < 0:
            raise ValueError("amount must be positive")
        self._balance += amount
        self._make_ledger_entry(amount, "deposit")


In [2]:
account = BankAccount("John", "Doe", "123456789", 100, False)

('2023-06-09 10:32:37.473293 account creation 100; balance 100',)


In [42]:
acc2 = BankAccount("Jenny", "Smith", "123456780", 100, False)

In [41]:
account is acc2

False

In [47]:
acc2.ledger

[]

In [20]:
account.__dict__

{'_first_name': 'John',
 '_last_name': 'Doe',
 '_account_number': '123456789',
 '_balance': 100,
 '_is_overdraft_allowed': False,
 '_ledger': []}

In [21]:
account

BankAccount(first_name='John', last_name='Doe', account_number='123456789', balance=100, is_overdraft_allowed=False)

In [82]:
account.withdraw(50)

('2023-06-09 10:10:24.574450 withdraw -50; balance 50',)


In [83]:
account.withdraw(30)

('2023-06-09 10:10:24.574450 withdraw -50; balance 50', '2023-06-09 10:10:31.139445 withdraw -30; balance 20')


In [36]:
account.deposit(100)

['2023-06-09 09:44:01.894936 withdraw -50; balance 50', '2023-06-09 09:44:12.994644 withdraw -20; balance 30', '2023-06-09 09:44:39.038206 deposit 100, balance 130']
